<a href="https://colab.research.google.com/github/nivedita-rajesh/Song-Reccomendation-System/blob/main/Song_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Recommender System using Apache Spark and Python

## Necessary Package Imports

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=98a0332f0e9082df303d3ffde084a9d5486f3fb19958a414dc7e6a61ed7ca5c3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


## Loading data

In [2]:
from pyspark import SparkContext
sc = SparkContext()

## MapReduce

In [ ]:
import random
lst=[]
genre=set()
with open("song.txt", "r") as file1:
    lines= file1.readlines()

for line in lines:
    lst.append(line.strip().replace('"',"").split(";"))


random.shuffle(lines)
lines = lines[50]
random.shuffle(lst)
lst=lst[1:51]

for record in lst:
  #print(record[-1])
  genre.add(record[-1])
# print(lst[:10])
print(genre)

In [ ]:
file2=open('new.txt','a')
for track in lst :
  genre = track[-1]
  if(genre in ['tone' ,'wonky',"modern classical","deep liquid", "bounce", "metal", "australian dance", "lo-fi"]):
    track.append('angry')
    file2.write(str(track)+"\n")

  elif("pop" in genre or genre in ["spanish hip hop","salsa"]  or "rock" in genre or "metal" in genre):
    track.append('happy')
    file2.write(str(track)+"\n")
  
  elif(genre in ['anime', "comedy rock" ,"boogie woogie","bubblegum dance","preverb", "chicago soul", "alternative emo", "pop emo"]):
    track.append('sad')
    file2.write(str(track)+"\n")

  elif(genre in ["edm", "composition d", "cool jazz", "christian music", "video game music"]):
    track.append("neutral")
    file2.write(str(track)+"\n")



In [60]:
#read from the user's listen history
redundant_lst= [['3rMyMv8EjKXoPnaRo2hdJN', "Stuntin' Like My Daddy - Street", 'Birdman, Lil Wayne', '5', 'pop rap','happy'],
['6lV2MSQmRIkycDScNtrBXO', 'Airplanes (feat. Hayley Williams of Paramore)', 'B.o.B, Hayley Williams', '1', 'pop rap','happy'],
['4hrae8atte6cRlSC9a7VCO', 'Always On Time', 'Ja Rule, Ashanti', '2', 'pop rap','happy'],
['03tqyYWC9Um2ZqU0ZN849H', 'No Hands [feat. Roscoe Dash and Wale]', 'Waka Flocka Flame', '3', 'pop rap','happy'],
['66TRwr5uJwPt15pfFkzhbi', 'Crank That (Soulja Boy)', 'Soulja Boy', '4', 'pop rap','angry'],
['6t2ubAB4iSYOuIpRAOGd4t', 'Cake', 'Flo Rida, 99 Percent', '8', 'pop rap','happy'],
['6t2ubAB4iSYOuIpRAOGd4t', 'Cake', 'Flo Rida, 99 Percent', '8', 'pop rap','happy'],
['66TRwr5uJwPt15mfFkzhbi', 'Crank That (Soulja Boy)', 'Soulja Boy', '4', 'pop rap','happy'],
['66TRwr5uJwPt15mfFqzhbi', 'Crank That (Soulja Boy)', 'Soulja Boy', '4', 'pop rap','angry'],
['66TRwr5uJwPt15mfFkzhbi', 'Crank That (Soulja Boy)', 'Soulja Boy', '4', 'pop rap','happy'],
['3rMyMv8EjKXoPnaRo2hlJN', "Stuntin' Like My Daddy - Street", 'Birdman, Lil Wayne', '5', 'pop rap','neutral'],
['3rMyMv8EjKXoPnaRl2hdJN', "Stuntin' Like My Daddy - Street", 'Birdman, Lil Wayne', '5', 'pop rap','angry'],
['3rMyMv8EjKXoPnaRo2hdJN', "Stuntin' Like My Daddy - Street", 'Birdman, Lil Wayne', '5', 'pop rap','happy'],
['7wqSzGeodspE3V6RBD5W8L', 'See You Again (feat. Charlie Puth)', 'Wiz Khalifa, Charlie Puth', '7', 'pop rap','happy'],
['6t2ubAB4iSYOuIpRAOGd4t', 'Cake', 'Flo Rida, 99 Percent', '8', 'pop rap','happy'],
['5IZc3KIVFhjzJ0L2kiXzUl', 'Promise', 'Kid Ink, Fetty Wap', '9', 'pop rap','happy'],
['3B7i9OKRRmIsSBHEbJz58Y', 'Grind With Me', 'Pretty Ricky', '10', 'pop rap','happy'],
['3tvWMBIblzT5FSjKtIeRR1', 'Whatever You Like', 'T.I.', '11', 'pop rap','happy'],
['1hWYT0w2R0J19rlVkiez7X', 'Battle Scars', 'Lupe Fiasco, Guy Sebastian', '12', 'pop rap','happy'],
['14B2bUopOga5V3ypld7d6n', 'Suga Suga', 'Baby Bash, Frankie J', '13', 'pop rap','happy'],
['3B7i9OKRRmIsSBHEbJz58Y', 'Grind With Me', 'Pretty Ricky', '10', 'pop rap','happy'],
['3B7i9OKRRmIsSBHEbJz58Y', 'Grind With Me', 'Pretty Ricky', '10', 'pop rap','happy'],
['14B2bUopOga5V3ypld7d6n', 'Suga Suga', 'Baby Bash, Frankie J', '13', 'pop rap','happy'],
['14B2bUopOga5V3ypld7dwn', 'SugewedSa', 'BabwrBash, Frankie J', '5', 'pop rap','sad'],
['14B2pUopOga5V3ypld7d6n', 'Suga Suga', 'Baby Bash, Frankie J', '13', 'pop rap','angry'],
['66TRwr5uJwPt15mfFkzhbi', 'Crank That (Soulja Boy)', 'Soulja Boy', '4', 'pop rap','happy']
]

redundant_lst

with open('dummy.txt','a') as f:
  for i in redundant_lst:
    f.write(str(i)+"\n")


In [61]:
from pyspark.sql import SparkSession

dataset = sc.textFile("dataset.txt")
a = dataset.map(lambda x:(x,1)).reduceByKey(lambda x,y : x + y).collect()

fin_list = []

for i in a:

  res = i[0].strip('][').replace("'","").split(', ') 
  if res[-3].isdigit():
    ans = "&".join(res[2:-2])

    while(not res[3].isdigit()):
      del res[3]
    res[2] = ans

  res.append(i[1])
  fin_list.append(res)

In [62]:
rdd = sc.parallelize(fin_list)
spark = SparkSession(sc)

print("Filtered Dataset")
schema = ["spotify_id","title","artist(s)","popularity","genre", "emotion", "count"]

rdd.toDF(schema).show()

Filtered Dataset
+--------------------+--------------------+--------------------+----------+-------+-------+-----+
|          spotify_id|               title|           artist(s)|popularity|  genre|emotion|count|
+--------------------+--------------------+--------------------+----------+-------+-------+-----+
|3rMyMv8EjKXoPnaRo...|"Stuntin Like My ...| Birdman&Lil Wayne&5|         5|pop rap|  happy|    2|
|66TRwr5uJwPt15pfF...|Crank That (Soulj...|        Soulja Boy&4|         4|pop rap|  angry|    1|
|66TRwr5uJwPt15mfF...|Crank That (Soulj...|        Soulja Boy&4|         4|pop rap|  happy|    3|
|3rMyMv8EjKXoPnaRl...|"Stuntin Like My ...| Birdman&Lil Wayne&5|         5|pop rap|  angry|    1|
|3B7i9OKRRmIsSBHEb...|       Grind With Me|     Pretty Ricky&10|        10|pop rap|  happy|    3|
|3tvWMBIblzT5FSjKt...|   Whatever You Like|             T.I.&11|        11|pop rap|  happy|    1|
|14B2bUopOga5V3ypl...|           Suga Suga|Baby Bash&Frankie...|        13|pop rap|  happy|    2|
|6l

In [63]:
rdd=rdd.filter(lambda x: x[-2]=="happy")
rdd.toDF(schema).show()


+--------------------+--------------------+--------------------+----------+-------+-------+-----+
|          spotify_id|               title|           artist(s)|popularity|  genre|emotion|count|
+--------------------+--------------------+--------------------+----------+-------+-------+-----+
|3rMyMv8EjKXoPnaRo...|"Stuntin Like My ...| Birdman&Lil Wayne&5|         5|pop rap|  happy|    2|
|66TRwr5uJwPt15mfF...|Crank That (Soulj...|        Soulja Boy&4|         4|pop rap|  happy|    3|
|3B7i9OKRRmIsSBHEb...|       Grind With Me|     Pretty Ricky&10|        10|pop rap|  happy|    3|
|3tvWMBIblzT5FSjKt...|   Whatever You Like|             T.I.&11|        11|pop rap|  happy|    1|
|14B2bUopOga5V3ypl...|           Suga Suga|Baby Bash&Frankie...|        13|pop rap|  happy|    2|
|6lV2MSQmRIkycDScN...|Airplanes (feat. ...|B.o.B&Hayley Will...|         1|pop rap|  happy|    1|
|4hrae8atte6cRlSC9...|      Always On Time|   Ja Rule&Ashanti&2|         2|pop rap|  happy|    1|
|03tqyYWC9Um2ZqU0Z..

In [70]:
print("Sorted Dataset")

rdd = rdd.sortBy(lambda x:x[-1], ascending=False)
rdd.toDF(schema).show()


Sorted Dataset
+--------------------+--------------------+--------------------+----------+-------+-------+-----+
|          spotify_id|               title|           artist(s)|popularity|  genre|emotion|count|
+--------------------+--------------------+--------------------+----------+-------+-------+-----+
|66TRwr5uJwPt15mfF...|Crank That (Soulj...|        Soulja Boy&4|         4|pop rap|  happy|    3|
|3B7i9OKRRmIsSBHEb...|       Grind With Me|     Pretty Ricky&10|        10|pop rap|  happy|    3|
|6t2ubAB4iSYOuIpRA...|                Cake|Flo Rida&99 Perce...|         8|pop rap|  happy|    3|
|3rMyMv8EjKXoPnaRo...|"Stuntin Like My ...| Birdman&Lil Wayne&5|         5|pop rap|  happy|    2|
|14B2bUopOga5V3ypl...|           Suga Suga|Baby Bash&Frankie...|        13|pop rap|  happy|    2|
|3tvWMBIblzT5FSjKt...|   Whatever You Like|             T.I.&11|        11|pop rap|  happy|    1|
|6lV2MSQmRIkycDScN...|Airplanes (feat. ...|B.o.B&Hayley Will...|         1|pop rap|  happy|    1|
|4hra

In [69]:
k=5
top_k = rdd.take(k)

for song in top_k:
  print(f"{song[1]} by {song[2]} has a popularity of {song[3]}. You have listened to it {song[-1]} time(s)")
  print(f"Listen song at https://open.spotify.com/track/{song[0]}")
  print()

Crank That (Soulja Boy) by Soulja Boy&4 has a popularity of 4. You have listened to it 3 time(s)
Listen song at https://open.spotify.com/track/66TRwr5uJwPt15mfFkzhbi

Grind With Me by Pretty Ricky&10 has a popularity of 10. You have listened to it 3 time(s)
Listen song at https://open.spotify.com/track/3B7i9OKRRmIsSBHEbJz58Y

Cake by Flo Rida&99 Percent&8 has a popularity of 8. You have listened to it 3 time(s)
Listen song at https://open.spotify.com/track/6t2ubAB4iSYOuIpRAOGd4t

"Stuntin Like My Daddy - Street" by Birdman&Lil Wayne&5 has a popularity of 5. You have listened to it 2 time(s)
Listen song at https://open.spotify.com/track/3rMyMv8EjKXoPnaRo2hdJN

Suga Suga by Baby Bash&Frankie J&13 has a popularity of 13. You have listened to it 2 time(s)
Listen song at https://open.spotify.com/track/14B2bUopOga5V3ypld7d6n

